In [1]:
from colmap_testing.colmap_helpers import read_write_model
import os
import glob
import cv2
import numpy as np

In [3]:
os.getcwd()

'c:\\Users\\Mathis\\Documents\\GitHub\\gaze_data_acquisition\\mathis'

In [4]:
path_to_model = os.path.join("colmap_testing", "colmap_ws", "dense","0","sparse")
paths_to_images = np.sort(glob.glob(os.path.join("colmap_testing", "colmap_ws", "dense", "0", "images")+ "/*.png"))

rgb_images = np.array([np.array(cv2.imread(imagepath)) for imagepath in paths_to_images])
colmap_model = read_write_model.read_model(path_to_model, ".bin")

#cameras:fx, fy, cx, cy, width, height
#images:camera_id, qvec, tvec, name 
#points:xyz, rgb, error, image_ids, point2D_idxs
cameras, images, points = read_write_model.read_model(path_to_model, ".bin")




In [5]:
# 3D point list with one line of data per point:
# POINT3D_ID, X, Y, Z, R, G, B, ERROR, TRACK[] as (IMAGE_ID, POINT2D_IDX)
random_point = points[3723]

# Image list with two lines of data per image:
#   IMAGE_ID, QW, QX, QY, QZ, TX, TY, TZ, CAMERA_ID, NAME
#   POINTS2D[] as (X, Y, POINT3D_ID)

first_image = images[1]

# Camera list with one line of data per camera:
#   CAMERA_ID, MODEL, WIDTH, HEIGHT, PARAMS[]
first_camera = cameras[1]



#The coordinates of the projection/camera center are given by -R^t * T, 
# where R^t is the inverse/transpose of the 3x3 rotation matrix composed from the quaternion and T is the translation vector. 
# The local camera coordinate system of an image is defined in a way that the X axis points to the right, 
# the Y axis to the bottom, and the Z axis to the front as seen from the image.

#calculate the projection/camera center




In [8]:
def get_frames(images, cameras, scale=1):
    frames = []
    for img in images.values():
        # rotation
        R = read_write_model.qvec2rotmat(img.qvec)

        # translation
        t = img.tvec

        # invert
        t = -R.T @ t
        R = R.T

        # intrinsics
        cam = cameras[img.camera_id]

        if cam.model in ("SIMPLE_PINHOLE", "SIMPLE_RADIAL", "RADIAL"):
            fx = fy = cam.params[0]
            cx = cam.params[1]
            cy = cam.params[2]
        elif cam.model in ("PINHOLE", "OPENCV", "OPENCV_FISHEYE", "FULL_OPENCV"):
            fx = cam.params[0]
            fy = cam.params[1]
            cx = cam.params[2]
            cy = cam.params[3]
        else:
            raise Exception("Camera model not supported")

        # intrinsics
        K = np.identity(3)
        K[0, 0] = fx
        K[1, 1] = fy
        K[0, 2] = cx
        K[1, 2] = cy
        frames.append((R, t, K))
    return frames

#frames are composed of R, t, K
frames = get_frames(images, cameras)

random_point = points[3723]
random_point_3d = random_point[1] #3d point coordinates xyz

In [9]:
#project 3d point using the camera projection matrix onto the rgb image
#TODO: make sure the frames are corresponding to the right images
for i, frame in enumerate(frames):
    R, t, K = frame
    P = K @ (R @ random_point_3d + t)
    P = P / P[2]
    print(i, P)
    cv2.circle(rgb_images[i], (int(P[0]), int(P[1])), 5, (0, 0, 255), -1)
    cv2.imshow("image", rgb_images[i])
    cv2.waitKey(0)
    cv2.destroyAllWindows()



0 [1.22576950e+03 4.50474446e+02 1.00000000e+00]
1 [952.43305569 282.97841354   1.        ]
2 [415.70233267 247.05284517   1.        ]
3 [952.40620853 237.56629154   1.        ]
4 [638.41064815 310.72016729   1.        ]
5 [998.61035567 322.47240154   1.        ]
6 [949.20136585 298.99415174   1.        ]
7 [294.51384336 216.05573849   1.        ]
8 [500.37323578 175.81385579   1.        ]
9 [595.87075354 250.98826724   1.        ]
10 [632.84356887 321.28506154   1.        ]
